In [1]:
import pandas as pd
import numpy as np
import re
import string
from textblob import TextBlob

In [2]:
from nltk.stem.snowball import SnowballStemmer

In [3]:
stemmer = SnowballStemmer("english")

In [4]:
pd.set_option('display.max_colwidth', -1)

C:\Users\Carretero\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [94]:
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [95]:
test = pd.read_csv("test.csv",encoding = "ISO-8859-1")
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Cuento los links de cada tweet: (tengo que hacerlo antes de limpiarlos porque en la limpieza se pierden)

In [96]:
train['cantidad_links'] = train['text'].str.count('http')

In [97]:
test['cantidad_links'] = test['text'].str.count('http')

In [98]:
train.nlargest(30,'cantidad_links').head()

,id,keyword,location,text,target,cantidad_links
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
7082,10143,upheaval,Attock,Ancient Mayan Tablet found via http://t.co/LmUMzkLtln http://t.co/yebxxAryBF http://t.co/SRRUqfffr6 http://t.co/CadzxAgMSI,0,4
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/GhmmUj7GbE (vÂÃ_ http://t.co/u9LvvLzhYe) http://t.co/LyJ57pq3yX,0,3
1373,1980,bush%20fires,Trinidad and Tobago,Drought fuels bush fires in Jamaica - http://t.co/ZDtDqQbAHC http://t.co/PsQCNsVfgP - @JamaicaObserver @cnewslive RE https://t.co/6ZGef8J8Bm,1,3


In [99]:
# Remuevo los links
train['text'] = train['text'].replace(r'http\S+', '', regex=True)
test['text'] = test['text'].replace(r'http\S+', '', regex=True)

## Cuento la cantidad de apariciones de caracteres inusuales

In [100]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
non_alphabetic_characters = ['\?','@','\!','\$']
mean_non_alphabetic_characters = [train[train['text'].str.contains(i)].target.mean() for i in non_alphabetic_characters]
times_non_alphabetic_characters_apears = [len(train[train['text'].str.contains(i)]) for i in non_alphabetic_characters]

In [101]:
print(non_alphabetic_characters)
print(mean_non_alphabetic_characters)
print(times_non_alphabetic_characters_apears)

['\\?', '@', '\\!', '\\$']
[0.27953890489913547, 0.3315350662089259, 0.27298050139275765, 0.3157894736842105]
[1041, 2039, 718, 38]


In [102]:
non_alphabetic_characters = ['?','@','!']

In [103]:
for i in non_alphabetic_characters:
    train[i]=train['text'].apply(lambda x: x.split(' ').count(i))
    test[i]=test['text'].apply(lambda x: x.split(' ').count(i))

In [104]:
# Si tienen un $ seguido de un numero

In [105]:
def money_related(text):
    money_related = False
    for i in range(len(text)-1):
        if(text[i]=='$' and text[i+1].isnumeric()):
            money_related = True
    return(money_related)

In [106]:
train['money_related']=train['text'].apply(lambda x: money_related(x))
test['money_related']=test['text'].apply(lambda x: money_related(x))

In [107]:
#Si tiene una palabra que son todas mayusculas

In [108]:
# Mentions country

In [109]:
# Detectar_fechas,horas,y años

In [110]:
def has_dates(text):
    has_dates = False
    for i in range(len(text)-2):
        if(text[i].isnumeric() and ((text[i+1]=="/") or (text[i+1]=="\\")or  (text[i+1]=="-")) and (text[i+2].isnumeric())):
            has_dates = True
    return(has_dates)

In [111]:
train['tiene_fecha'] = train['text'].apply(lambda x: has_dates(x))
test['tiene_fecha'] = test['text'].apply(lambda x: has_dates(x))

In [112]:
def has_year(text):
    has_year = False
    for i in range(len(text)-4):
        if(text[i]=='2' and text[i+1]=='0' and text[i+2].isnumeric() and text[i+3].isnumeric()):
            has_year = True
        elif(text[i]=='1' and text[i+1]=='9' and text[i+2].isnumeric() and text[i+3].isnumeric()):
            has_year = True
    return(has_year)

In [113]:
train['has_year'] = train['text'].apply(lambda x: has_year(x))
test['has_year'] = test['text'].apply(lambda x: has_year(x))

In [114]:
def otra_forma(text):
    tiene_hora = False
    for i in text:
        if (i.isnumeric()):
            tiene_hora = True
    if(tiene_hora and ( (text.count('PM')>0) or (text.count('AM')>0))   ):
        tiene_hora = True
    else:
        tiene_hora = False
    return(tiene_hora)

In [115]:
def has_time(text):
    has_time = False
    for i in range(len(text)-1):
        if(text[i].isnumeric() and text[i+1]==":"):
            has_time = True
    return(has_time)

In [116]:
train['tiene_hora']=train['text'].apply(lambda x: has_time(x))
test['tiene_hora']=test['text'].apply(lambda x: has_time(x))

In [117]:
train['tiene_hora_2'] = train['text'].apply(lambda x:otra_forma(x))
test['tiene_hora_2'] = test['text'].apply(lambda x:otra_forma(x))

In [118]:
# numeros

In [119]:
def contador_de_numeros(text):
    cantidad = 0
    #primero me aseguro que los numeros no sean fechas u horas 
    
    
    for i in range(0,10):
        if (text.count(str(i))>0):
            cantidad = cantidad+1
    return(cantidad)

In [120]:
#train['numeros'] = train['text'].apply(lambda x: contador_de_numeros(x))

In [121]:
#train['numeros'] = train[(train['numeros']>0) & (train['tiene_fecha']==False) & (train['tiene_fecha_rara']==False) & (train['tiene_hora']==False)].head(50)

# Contador de hashtags

In [122]:
def hashtags_counter(text):
    text = text.lower()
    words = text.split(' ')
    hashtags = []
    for i in words:
        if i.startswith('#'):
            hashtags.append(i)
    return(' '.join(hashtags))

In [123]:
train['hashtags'] = train['text'].apply(lambda x: hashtags_counter(x))
test['hashtags'] = test['text'].apply(lambda x: hashtags_counter(x))

In [124]:
hashtags_por_separado = train.copy()
hashtags_por_separado.set_index(['target'],inplace=True)
hashtags_por_separado = hashtags_por_separado['hashtags'].str.split(expand=True).stack().to_frame()
hashtags_por_separado.columns = ['hashtag']
hashtags_por_separado = hashtags_por_separado.reset_index()
hashtags_por_separado = hashtags_por_separado[['hashtag','target']]
hashtags_por_separado.head()

,hashtag,target
0,#earthquake,1
1,#wildfires,1
2,#alaska,1
3,#wildfires,1
4,#rockyfire,1


In [125]:
hashtag_mas_comunes = hashtags_por_separado.groupby('hashtag').agg({'target':['mean','sum']})
hashtag_mas_comunes.columns = ['promedio_target','cantidad_apariciones']
hashtag_mas_comunes = hashtag_mas_comunes[(hashtag_mas_comunes['promedio_target']>0.6) | (hashtag_mas_comunes['promedio_target']<0.4)]
hashtag_mas_comunes = hashtag_mas_comunes.nlargest(50,'cantidad_apariciones')
hashtag_mas_comunes

,promedio_target,cantidad_apariciones
hashtag,,
#news,0.738462,48
#???,1.000000,23
#hiroshima,1.000000,21
#earthquake,1.000000,19
#??,0.947368,18
#india,1.000000,10
#japan,1.000000,10
#abstorm,1.000000,9
#sismo,0.900000,9


In [126]:
for i in hashtag_mas_comunes.index.values:
    train[i]= train['text'].apply(lambda x: x.count(i))
    test[i]= test['text'].apply(lambda x: x.count(i))

In [127]:
train=train.drop(columns=['hashtags'])
test=test.drop(columns=['hashtags'])

## Limpieza de los textos

In [128]:
train_limpio = train.copy()

In [129]:
test_limpio = test.copy()

In [130]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()  #quito_mayusculas
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('target' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    text = re.sub("[^0-9a-zA-Z]+", " ", text)
    return text

In [131]:
test_limpio['text'] = test_limpio['text'].apply(lambda x: clean_text(x))
train_limpio['text'] = train_limpio['text'].apply(lambda x: clean_text(x))
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#japã_n,#mh370,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,0
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,0
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,0
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,0


ANALIZO CUANTOS KEYWORDS CONTIENE CADA TWEET, TENIENDO EN CUENTA TODOS LOS KEYWORDS POSIBLES

In [132]:
dic = list(dict.fromkeys(train_limpio['keyword'].to_list()))
dic.pop(0)
for word in dic:
    index = dic.index(word)
    dic[index] = word.replace('%20',' ')


In [133]:
import re
def suma_de_keywords(lista,string):
    x= 0
    for word in lista:
        x += 1 if (word in string.lower()) else 0
    return x


In [134]:
train_limpio['cant_keywords'] = (train_limpio['text'].apply(lambda x: x.lower())).apply(lambda x: suma_de_keywords(dic,x))
test_limpio['cant_keywords'] = (test_limpio['text'].apply(lambda x: x.lower())).apply(lambda x: suma_de_keywords(dic,x))



In [135]:
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#mh370,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,cant_keywords
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,2
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,1
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,3
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,0,3


In [136]:
test_limpio.head()

,id,keyword,location,text,cantidad_links,?,@,!,money_related,tiene_fecha,...,#mh370,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,cant_keywords
0,0,NaN,NaN,just happened a terrible car crash,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,0,1
1,2,NaN,NaN,heard about earthquake is different cities stay safe everyone,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,0,1
2,3,NaN,NaN,there is a forest fire at spot pond geese are fleeing across the street i cannot save them all,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,0,2
3,9,NaN,NaN,apocalypse lighting spokane wildfires,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,0,3
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,0,1


## Analizar las repeticiones de un tweet

In [137]:
test_limpio['repeticiones'] = test_limpio.groupby('text')['id'].transform('count')
train_limpio['repeticiones'] = train_limpio.groupby('text')['id'].transform('count')

In [138]:
train_limpio.nlargest(20,'repeticiones')

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,cant_keywords,repeticiones
5876,8393,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5877,8394,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5879,8397,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5881,8400,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5882,8401,sandstorm,hkXfYMhEx,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5883,8402,sandstorm,US,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5884,8404,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5885,8405,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5888,8409,sandstorm,United States,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24
5889,8410,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,24


## Voy a dejar los keywords con stemming

### Con esto busco que los keywords me ayuden a decidir el target pero solo lo hagan si hay "suficientes" como para hacerlo.

In [139]:
rowsWithNonNullKeyword = train_limpio.loc[train_limpio.keyword.notnull()]
rowsWithNonNullKeyword[:4]

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,cant_keywords,repeticiones
31,48,ablaze,Birmingham,bbcmtd wholesale markets ablaze,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,1
32,49,ablaze,Est. September 2012 - Bristol,we always try to bring the heavy metal rt,0,1,0,0,0,False,...,0,0,0,0,0,0,0,0,0,1
33,50,ablaze,AFRICA,africanbaze breaking newsnigeria flag set ablaze in aba,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,2,1
34,52,ablaze,"Philadelphia, PA",crying out for more set me ablaze,0,0,0,0,0,False,...,0,0,0,0,0,0,0,0,2,1


In [140]:
keywordsCount = rowsWithNonNullKeyword.groupby("keyword").agg({"id": "count"}).reset_index()
keywordsCount.columns = ["keyword", "count"]
keywordsCount.head(2)

,keyword,count
0,ablaze,36
1,accident,35


In [141]:
train_limpio['keyword'] = train_limpio['keyword'].replace({ np.nan:'null'})
train_limpio["keyword"] = train_limpio['keyword'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [142]:
test_limpio['keyword'] = test_limpio['keyword'].replace({ np.nan:'null'})
test_limpio["keyword"] = test_limpio['keyword'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [143]:
keywords = train_limpio.groupby('keyword').agg({'target':['count','mean']})
keywords.columns=['cantidad','target_mean']
keywords

,cantidad,target_mean
keyword,,
ablaz,36,0.361111
accid,35,0.685714
aftershock,34,0.000000
airplane%20accid,35,0.857143
ambul,38,0.526316
...,...,...
wildfir,33,0.878788
windstorm,40,0.400000
wound,70,0.514286


In [144]:
keywords[keywords['cantidad']>20]

,cantidad,target_mean
keyword,,
ablaz,36,0.361111
accid,35,0.685714
aftershock,34,0.000000
airplane%20accid,35,0.857143
ambul,38,0.526316
...,...,...
wildfir,33,0.878788
windstorm,40,0.400000
wound,70,0.514286


In [145]:
# Me armo una columna por cada keyword, no esta bien pero bueno

In [146]:
for i in keywords.index:
    train_limpio[i+'-'+'keyword'] = train_limpio['keyword'].str.contains(i)
    test_limpio[i+'-'+'keyword'] = test_limpio['keyword'].str.contains(i)
# Tengo que agregar el +'-'+'keyword' o sino algunas palabras pueden ser sobreescritas al vectorizar los textos

# Limpio las locations:

In [147]:
train_limpio['location'] = train_limpio['location'].apply(lambda x: str(x).lower())
test_limpio['location'] = test_limpio['location'].apply(lambda x: str(x).lower())
train_limpio['location'] = train_limpio['location'].apply(lambda x: clean_text(x))
test_limpio['location'] = test_limpio['location'].apply(lambda x: clean_text(x))

## Sabemos que hay locations repetidas pero con distinto nombre, veo si puedo agrupar

In [149]:
rowsWithNonNullLocation = train_limpio.loc[train_limpio.location.notnull()].copy()
rowsWithNonNullLocation

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,volcano-keyword,war%20zon-keyword,weapon-keyword,whirlwind-keyword,wild%20fir-keyword,wildfir-keyword,windstorm-keyword,wound-keyword,wreck-keyword,wreckag-keyword
0,1,null,nan,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
1,4,null,nan,forest fire near la ronge sask canada,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
2,5,null,nan,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
3,6,null,nan,people receive wildfires evacuation orders in california,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
4,7,null,nan,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,null,nan,two giant cranes holding a bridge collapse into nearby homes,1,1,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
7609,10870,null,nan,ariaahrary thetawniest the out of control wild fires in california even in the northern part of the state very troubling,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
7610,10871,null,nan,s of volcano hawaii,1,1,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
7611,10872,null,nan,police investigating after an ebike collided with a car in little portugal ebike rider suffered serious nonlife threatening injuries,1,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False


In [150]:
locationCount = rowsWithNonNullLocation.groupby("location").agg({"id":"count"}).reset_index()
locationCount.columns = ["location", "count"]
locationCount.loc[locationCount["count"] > 4]

,location,count
0,,92
1,,39
74,road to the billionaires club,6
82,t,20
127,africa,5
...,...,...
2876,washington dc,41
2967,wisconsin,5
2981,world,12
2983,world wide,5


### Casos como New York, new york, ny y nyc nos pueden molestar, veo si hay manera de arreglarlo

In [151]:
#list(locationCount.loc[locationCount["count"] > 4]["location"])

#Asheville, NC | Atlanta | Atlanta, GA | Austin, TX | Australia | Boston, MA | Brasil | Brooklyn, NY | Calgary, AB | Calgary, Alberta
#California | California, USA | California, United States | Canada | Charlotte, NC | Chicago | Chicago, IL | Cleveland, OH
#Colorado | Coventry | Dallas, TX | Denver, CO | Denver, Colorado | Earth | Everywhere | Florida | Florida, USA
#Global | Houston, TX | India | Indiana | Indonesia | Ireland | Kenya | Lagos | Lagos, Nigeria | Leeds, England
#London | London, England | London, UK | Los Angeles | Los Angeles, CA | Manchester | Massachusetts
#Melbourne, Australia | Memphis, TN | Morioh, Japan | Mumbai | NY | NYC | Nashville, TN | New Hampshire
#New Jersey | New York | New York City | New York, NY | New York, USA | Newcastle | Nigeria | North Carolina
#Oklahoma City, OK | Orland, FL | Pakistan | Paterson, New Jersey | Pedophile hunting ground | Pennsylvania
#Pennsylvania, USA | Philippines | Planet Earth | Portland, OR | Puerto Rico | Sacramento, CA | San Diego, CA
# San Francisco | San Francisco, CA | San Jose, CA | Scotland | Seattle | Seattle, WA | Singapore | South Africa
# Southern California | Sydney | Tampa, FL | Tennessee | Texas | Texas, USA | The Netherlands | Toronto | UK
# US | USA | United Kingdom | United States | Vancouver, BC | Washington, D.C. | Washington, DC | Winsconsin
# World | WorldWide | Worldwide | in the Word of God | ss | worldwide

In [152]:
realPlaces = {"NC": "NC", "Atlanta": "Atlanta", "TX": "TX", "Texas": "TX", "Australia": "Australia", "Boston": "MA", "MA": "MA", "Brasil": "Brasil", "Brooklyn": "NY", "Calgary": "AB", "AB": "AB", "California": "California", "Canada": "Canada", "Chicago": "Chicago", "Cleveland": "Cleveland", "Colorado": "Colorado", "Coventry": "Coventry", "Dallas": "TX", "Denver": "CO", "CO": "CO", "Colorado": "CO", "Florida": "Florida", "Houston": "TX", "India": "India", "Indiana": "Indiana", "Indonesia": "Indonesia", "Ireland": "Ireland", "Kenya": "Kenya", "Lagos": "Lagos", "Leeds": "UK", "London": "UK", "UK": "UK", "Los Angeles": "Los Angeles", "Manchester": "Manchester", "Massachusetts": "Massachusetts", "TN": "TN", "Japan": "Japan", "Mumbai": "Mumbai", "NY": "NY", "NYC": "NYC", "New Hampshire": "New Hampshire", "New Jersey": "New Jersey", "New York": "NY", "New York City": "NYC", "USA": "USA", "Newcastle": "Newcastle", "Nigeria": "Nigeria", "North Carolina": "NC", "Oklahoma City": "Oklahoma City", "Orland": "Orland", "Pakistan": "Pakistan", "Pennsylvania": "Pennsylvania", "Philippines": "Philippines", "Portland": "Portland", "Puerto Rico": "Puerto Rico", "Sacramento": "Sacramento", "San Diego": "San Diego", "San Francisco": "San Francisco", "San Jose": "San Jose", "Scotland": "Scotland", "Seattle": "Seattle", "Singapore": "Singapore", "South Africa": "South Africa", "Southern California": "Southern California", "Sydney": "Sydney", "Tampa": "Tampa", "Tennesse": "Tennesse", "Texas": "TX", "The Netherlands": "The Netherlands", "Toronto": "Toronto", "UK": "UK", "US": "USA", "USA": "USA", "United Kingdom": "UK", "United States": "USA", "Vancouver": "Vancouver", "Washington": "Washington", "Winsconsin": "Winsconsin"}                                                                                                                                 
realPlacesWithSpaces = ["Los Angeles", "New Hampshire", "New Jersey", "New York", "North Carolina", "Oklahoma City", "Puerto Rico", "San Diego", "San Francisco", "San Jose", "South Africa", "Southern California", "The Netherlands", "United Kingdom", "United States"]
notRealPlaces = ["Earth", "Everywhere", "Global", "Pedophile hunting ground", "Planet Earth", "World", "WorldWide", "Worldwide", "in the Word of God", "ss", "worldwide"]

In [153]:
def getLocationGroup(location):
    if location in notRealPlaces:
        return location
    else:
        for place in realPlacesWithSpaces:
            if(place in location):
                return realPlaces[place]
        locationString = location.replace(",", "")
        for word in locationString.split(" "):
            if word in realPlaces:
                return realPlaces[word]
        return re.escape(location)
    
def isRealPlace(location):
    return getLocationGroup(location) in realPlaces

train_limpio["location"] = train_limpio["location"].replace({ np.nan:'null'})
train_limpio["location"] = train_limpio["location"].map(lambda x: getLocationGroup(x))
train_limpio["isRealPlace"] = train_limpio["location"].map(lambda x: isRealPlace(x))

test_limpio["location"] = test_limpio["location"].replace({ np.nan:'null'})
test_limpio["location"] = test_limpio["location"].map(lambda x: getLocationGroup(x))
test_limpio["isRealPlace"] = test_limpio["location"].map(lambda x: isRealPlace(x))

In [154]:
locations = train_limpio.groupby('location').agg({'target':['count','mean']})
locations.columns=['cantidad','target_mean']
locations

,cantidad,target_mean
location,,
,92,0.304348
\,39,0.461538
\ access\ rt\ link\ please\ x,1,0.000000
\ act,1,0.000000
\ ag,1,0.000000
...,...,...
zboyerwashingtontimescom,1,1.000000
zeerust\ south\ africa,1,0.000000
zero\ branco,1,0.000000


In [155]:
for i in locations.index:
    train_limpio[i+'-'+'location'] = train_limpio['location'].str.contains(i)
    test_limpio[i+'-'+'location'] = test_limpio['location'].str.contains(i)

# Remuevo informacion
## Remuevo el ID

In [209]:
train_limpio = train_limpio.drop(columns=['id'])
test_limpio = test_limpio.drop(columns=['id'])

In [157]:
train_keywords = train_limpio.pop("keyword")
train_locations = train_limpio.pop("location")
test_keywords = test_limpio.pop("keyword")
test_locations = test_limpio.pop("location")

In [158]:
test_limpio.head()

,text,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,tiene_hora,tiene_hora_2,...,youre\ not\ forever\ -location,yuba\ city\ ca-location,yulee\ fl-location,yuukosans\ shop-location,zac\ newsome\ loves\ me-location,zboyerwashingtontimescom-location,zeerust\ south\ africa-location,zero\ branco-location,ziam\ af\ -location,zimbabwe-location
0,just happened a terrible car crash,0,0,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,heard about earthquake is different cities stay safe everyone,0,0,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,there is a forest fire at spot pond geese are fleeing across the street i cannot save them all,0,0,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,apocalypse lighting spokane wildfires,0,0,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,typhoon soudelor kills in china and taiwan,0,0,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Remuevo los datos repetidos
### No remuevo los repetivos del test obviamente
# Al final los dejo, da un poco mejor conservando los datos repetidos

In [159]:
train_limpio.shape

(7613, 3263)

In [160]:
test_limpio.shape

(3263, 3262)

In [161]:
#train_limpio = train_limpio.drop_duplicates()
#train_limpio.shape

In [162]:
train_limpio.head()

,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,tiene_hora,...,youre\ not\ forever\ -location,yuba\ city\ ca-location,yulee\ fl-location,yuukosans\ shop-location,zac\ newsome\ loves\ me-location,zboyerwashingtontimescom-location,zeerust\ south\ africa-location,zero\ branco-location,ziam\ af\ -location,zimbabwe-location
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,people receive wildfires evacuation orders in california,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Longitud de los tweets

In [163]:
import math

def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [164]:
train_limpio['longitud_tweet'] = train_limpio['text'].apply(lambda x: roundup(len(x)))

In [165]:
train_limpio.head(2)

,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,tiene_hora,...,yuba\ city\ ca-location,yulee\ fl-location,yuukosans\ shop-location,zac\ newsome\ loves\ me-location,zboyerwashingtontimescom-location,zeerust\ south\ africa-location,zero\ branco-location,ziam\ af\ -location,zimbabwe-location,longitud_tweet
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,70
1,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,40


In [166]:
test_limpio['longitud_tweet'] = test_limpio['text'].apply(lambda x: roundup(len(x)))

## Contar la cantidad de veces seguidas que se repite una letra, si alguna se repite mas de 3 veces, es probable que haya sido intencional, lo cual le quita credibildad

In [167]:
def letras_repetidas(text):
    max_repeteated_letters = 0
    current_repeated_letters = 1
    for word in text:
        if(current_repeated_letters>max_repeteated_letters):
            max_repeteated_letters=current_repeated_letters
        current_repeated_letters=1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeteated_letters):
                    max_repeteated_letters=current_repeated_letters
                current_repeated_letters=1
    return (max_repeteated_letters)

In [168]:
def remover_palabras_con_letras_repetidas(text):
    max_repeated_letters = 0
    current_repeated_letters = 1
    new_text = []
    for word in text:
        current_repeated_letters=1
        max_repeated_letters = 1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeated_letters):
                    max_repeated_letters=current_repeated_letters
                current_repeated_letters=1
        if(current_repeated_letters<=2 and max_repeated_letters<=2):
            new_text.append(word)  #remuevo las palabras que tengan 3 o mas letras seguidas repetidas
    return ( ' '.join(new_text))

In [169]:
train_limpio['letras_seguidas'] = train_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

test_limpio['letras_seguidas'] = test_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

In [170]:
letras_seguidas_train = train_limpio['letras_seguidas']
letras_seguidas_test = test_limpio['letras_seguidas']

In [171]:
#Remuevo esas palabras, x ahora las dejo
train_limpio['text'] = train_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
test_limpio['text'] = test_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
train_limpio.head(2)

,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,tiene_hora,...,yulee\ fl-location,yuukosans\ shop-location,zac\ newsome\ loves\ me-location,zboyerwashingtontimescom-location,zeerust\ south\ africa-location,zero\ branco-location,ziam\ af\ -location,zimbabwe-location,longitud_tweet,letras_seguidas
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,70,2
1,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,40,1


### Parece dar buenos resultados pero obviamente no reemplaza un programa que me detecte cualquier error ortografico

# Correccion de typos

### Lo deje comentado por ser demasiado lento

In [172]:
#Prueba basica
#pruebita = train_limpio.iloc[1:150].copy()

def spell_correcter(text):
    word_list = text.split(' ')
    return [str(TextBlob(word).correct()) for word in word_list]

from textblob import TextBlob
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

#pruebita['text_2'] = pruebita['text'].apply(lambda x: ' '.join(x))
#pruebita.head()

In [173]:
#Confirmé que funciona, lo aplico a todas

In [174]:
#train_limpio['text'] = train_limpio['text'].apply(lambda x: spell_correcter(x))

#train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: spell_correcter(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: ' '.join(x))

In [175]:
#train_limpio.to_csv('train_con_textos_corregidos.csv', index=False)

#test_limpio.to_csv('test_con_textos_corregidos.csv', index=False)

In [176]:
#train_texto_corregido = pd.read_csv("train_con_textos_corregidos.csv",encoding = "ISO-8859-1")
#test_texto_corregido = pd.read_csv("test_con_textos_corregidos.csv",encoding = "ISO-8859-1")

In [177]:
#train_texto_corregido = train_texto_corregido.text.fillna(0)

In [178]:
#test_texto_corregido = test_texto_corregido.text.fillna(0)

In [179]:
#train_limpio['texto_corregido'] = train_texto_corregido

In [180]:
#test_limpio['texto_corregido'] = test_texto_corregido

In [181]:
#train_limpio[train_limpio['texto_corregido']!=train_limpio['text']].head()

In [182]:
#Corrige typos que no deberia, mejor no lo uso

# Analisis de sentimiento y de objetividad

In [183]:
b = TextBlob("Simple is better than complex")
b.sentiment

Sentiment(polarity=0.06666666666666667, subjectivity=0.41904761904761906)

In [184]:
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))
pruebita = train_limpio.iloc[1:150].copy()
pruebita['analisis_sentimientos'] = pruebita['text'].apply(lambda x: (TextBlob(str(x)).sentiment))
pruebita.head()

,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,tiene_hora,...,yuukosans\ shop-location,zac\ newsome\ loves\ me-location,zboyerwashingtontimescom-location,zeerust\ south\ africa-location,zero\ branco-location,ziam\ af\ -location,zimbabwe-location,longitud_tweet,letras_seguidas,analisis_sentimientos
1,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,40,1,"(0.1, 0.4)"
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,130,2,"(-0.018750000000000003, 0.3875)"
3,people receive wildfires evacuation orders in california,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,60,1,"(0.0, 0.0)"
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,90,2,"(0.0, 0.0)"
5,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,100,1,"(-0.1125, 0.2375)"


In [185]:
train_limpio['sentimiento'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
train_limpio['objetividad'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [186]:
test_limpio['sentimiento'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
test_limpio['objetividad'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

# Stemming

In [187]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [188]:
import string

def remove_punctuation(text):
    '''a function for removing punctuation'''
    # replacing the punctuations with no space.
    # which in effect deletes the punctuation marks.
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks.
    return text.translate(translator)



In [189]:
test_limpio['text'] = test_limpio['text'].apply(remove_punctuation)
train_limpio['text'] = train_limpio['text'].apply(remove_punctuation)

In [190]:
# create an object of stemming function.
stemmer = SnowballStemmer("english")

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 



In [191]:
train_limpio['text'] = train_limpio['text'].apply(stemming)
test_limpio['text'] = test_limpio['text'].apply(stemming)

In [192]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word',max_features=4500 ,binary=True)
vectorizer.fit(train_limpio['text'])
new_train = vectorizer.transform(train_limpio['text']).todense()

In [193]:
new_train = pd.DataFrame(new_train, columns=vectorizer.get_feature_names())

In [194]:
train_text = train_limpio.pop('text')

In [195]:
train_limpio = train_limpio.reset_index(drop=True)

In [196]:
for i in train_limpio.columns:
    new_train[i]=train_limpio[i]

In [197]:
#Ahora lo hago con test
new_test = vectorizer.transform(test_limpio['text']).todense()
new_test = pd.DataFrame(new_test, columns=vectorizer.get_feature_names())

In [198]:
test_text = test_limpio.pop('text')

In [199]:
test_limpio = test_limpio.reset_index(drop=True)

In [200]:
for i in test_limpio.columns:
    new_test[i]=test_limpio[i]

## Guardo los csvs limpios para mas adelante

In [202]:
new_train.to_csv('train_limpio_con_BOW_de_5000_y_Stemming.csv', index=False)

In [203]:
new_test.to_csv('test_limpio_con_BOW_de_5000_y_Stemming.csv', index=False)

In [292]:
new_train_withNoDroppedColumns = new_train
new_test_withNoDroppedColumns = new_test

new_train_withNoDroppedColumns["text"] = train_text
new_test_withNoDroppedColumns["text"] = test_text

new_train_withNoDroppedColumns["keyword"] = train_keywords
new_train_withNoDroppedColumns["location"] = train_locations
new_test_withNoDroppedColumns["keyword"] = test_keywords
new_test_withNoDroppedColumns["location"] = test_locations

In [293]:
new_train_withNoDroppedColumns.to_csv('train_limpio_con_BOW_de_5000_y_Stemming_noDrops.csv', index=False)
new_test_withNoDroppedColumns.to_csv('test_limpio_con_BOW_de_5000_y_Stemming_noDrops.csv', index=False)